In [39]:
import os
import warnings

import findfile
from pyabsa import ABSAInstruction as absa_instruction

warnings.filterwarnings("ignore")
import pandas as pd

In [51]:
model_out_path = 'checkpoints/absa/laptop14'

In [41]:
id_train_file_path = '../datasets'
id_test_file_path = '../datasets'

id_tr_df = absa_instruction.data_utils.read_json(id_train_file_path, "laptop_quad_train.tsv")
id_te_df = absa_instruction.data_utils.read_json(id_test_file_path, "laptop_quad_test.tsv")

../datasets\laptop_quad_train.tsv.jsonl
../datasets\laptop_quad_test.tsv.jsonl


In [42]:
id_tr_df = pd.DataFrame(id_tr_df)
id_te_df = pd.DataFrame(id_te_df)

In [45]:
id_tr_df

,text,labels
0,acer wants $ 170 to just look at it then add t...,"[{'aspect': 'acer', 'opinion': 'NULL', 'polari..."
1,update : i repaired it myself for $ 12 .,"[{'aspect': 'NULL', 'opinion': 'NULL', 'polari..."
2,i had nothing to lose since it was a paper wei...,"[{'aspect': 'NULL', 'opinion': 'NULL', 'polari..."
3,the shame of it is knowing it took me 15 minut...,"[{'aspect': 'acer', 'opinion': 'NULL', 'polari..."
4,first one that they shipped was obviously defe...,"[{'aspect': 'NULL', 'opinion': 'defective', 'p..."
...,...,...
2929,i think the sound could be better .,"[{'aspect': 'sound', 'opinion': 'could be bett..."
2930,i didn ' t want to spend much on this as it is...,"[{'aspect': 'performs', 'opinion': 'better', '..."
2931,"an awesome product , well built - well worth y...","[{'aspect': 'product', 'opinion': 'awesome', '..."
2932,backlit keyboard is great ; feels sturdy ; fas...,"[{'aspect': 'backlit keyboard', 'opinion': 'gr..."


In [46]:
loader = absa_instruction.data_utils.InstructDatasetLoader(id_tr_df, id_te_df)

In [47]:
if loader.train_df_id is not None:
    loader.train_df_id = loader.prepare_instruction_dataloader(loader.train_df_id)
if loader.test_df_id is not None:
    loader.test_df_id = loader.prepare_instruction_dataloader(loader.test_df_id)
if loader.train_df_ood is not None:
    loader.train_df_ood = loader.prepare_instruction_dataloader(loader.train_df_ood)
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.prepare_instruction_dataloader(loader.test_df_ood)

In [48]:
model_checkpoint = "kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined"
t5_exp = absa_instruction.model.T5Generator(model_checkpoint)


[2025-03-04 21:54:43] (2.4.1.post1) ********** Available ACOS model checkpoints for Version:2.4.1.post1 (this version) **********
[2025-03-04 21:54:43] (2.4.1.post1) ********** Available ACOS model checkpoints for Version:2.4.1.post1 (this version) **********
[2025-03-04 21:54:43] (2.4.1.post1) Checkpoint:kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined is not found, you can raise an issue for requesting shares of checkpoints
[2025-03-04 21:54:43] (2.4.1.post1) No checkpoint found in Model Hub for task: kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined


In [49]:
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.create_datasets(
    t5_exp.tokenize_function_inputs
)

Map: 100%|██████████| 3075/3075 [00:00<00:00, 17194.01 examples/s]


In [50]:
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds

(DatasetDict({
     train: Dataset({
         features: ['text', 'labels'],
         num_rows: 10970
     })
     test: Dataset({
         features: ['text', 'labels'],
         num_rows: 3075
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['text', 'labels', 'input_ids', 'attention_mask'],
         num_rows: 10970
     })
     test: Dataset({
         features: ['text', 'labels', 'input_ids', 'attention_mask'],
         num_rows: 3075
     })
 }),
 None,
 None)

In [55]:
training_args = {
    "output_dir": model_out_path,
    "evaluation_strategy": "epoch",
    "save_strategy": "epoch",
    "learning_rate": 5e-5,
    "per_device_train_batch_size": 4,
    "per_device_eval_batch_size": 16,
    "num_train_epochs": 16,
    "weight_decay": 0.01,
    "warmup_ratio": 0.1,
    "load_best_model_at_end": True,
    "push_to_hub": False,
    "eval_accumulation_steps": 1,
    "predict_with_generate": True,
    "logging_steps": 1000000000,
    "use_mps_device": False,
    'fp16': False,
}

model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

1.4.0
